In [46]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [47]:
from graphdatascience import GraphDataScience 

gds = GraphDataScience( 
    os.environ[ "NEO4J_URI" ], 
    auth=(os.environ[ "NEO4J_USERNAME" ], os.environ[ "NEO4J_PASSWORD" ]) 
)

In [48]:
gds.graph.drop("communities")
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [49]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

Component count: 1
Component distribution: {'min': 24, 'p5': 24, 'max': 24, 'p999': 24, 'p99': 24, 'p1': 24, 'p10': 24, 'p90': 24, 'p50': 24, 'p25': 24, 'p75': 24, 'p95': 24, 'mean': 24.0}


In [50]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

writeMillis                                                             20
nodePropertiesWritten                                                   24
ranLevels                                                                2
didConverge                                                           True
nodeCount                                                               24
communityCount                                                           4
communityDistribution    {'min': 3, 'p5': 3, 'max': 11, 'p999': 11, 'p9...
modularity                                                        0.441563
modularities                       [0.3677867056245435, 0.441563184806428]
postProcessingMillis                                                     2
preProcessingMillis                                                      1
computeMillis                                                           29
configuration            {'writeProperty': 'communities', 'theta': 0.01...
Name: 0, dtype: object

In [51]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

[{'count(*)': 48}]

In [52]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:HAS_ENTITY]-(d:__Chunk__)
WITH c, count(distinct d) AS rank
SET c.rank = rank;
""")

[]

In [53]:
import pandas as pd
import numpy as np

community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

,Level,Number of communities,25th Percentile,50th Percentile,75th Percentile,90th Percentile,99th Percentile,Max
0,0,7,3.0,3.0,3.5,4.8,5.88,6
1,1,4,3.0,5.0,8.0,9.8,10.88,11


In [54]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")
community_info

[{'communityId': '0-3',
  'nodes': [{'id': '保險金受益人', 'description': None, 'type': 'Person'},
   {'id': '宣告利率', 'description': None, 'type': 'Concept'},
   {'id': '儲存生息', 'description': None, 'type': 'Financial_strategy'},
   {'id': '要保人', 'description': None, 'type': 'Person'},
   {'id': '本公司', 'description': None, 'type': 'Organization'},
   {'id': '增值回饋分享金',
    'description': '限以購買增額繳清保險金額辦理',
    'type': 'Financial_product'}],
  'rels': [{'start': '增值回饋分享金',
    'description': None,
    'type': 'BENEFITS',
    'end': '要保人'},
   {'start': '增值回饋分享金', 'description': None, 'type': 'PROVIDES', 'end': '本公司'},
   {'start': '增值回饋分享金',
    'description': None,
    'type': 'BENEFITS',
    'end': '保險金受益人'},
   {'start': '增值回饋分享金', 'description': None, 'type': 'BENEFITS', 'end': '要保人'},
   {'start': '增值回饋分享金',
    'description': None,
    'type': 'CALCULATES_WITH',
    'end': '宣告利率'},
   {'start': '要保人',
    'description': None,
    'type': 'TERMINATE_CONTRACT_WITH',
    'end': '本公司'},
   {'st

In [55]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [56]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

community_template = """請根據提供同社區的資訊包含 nodes 與 relationships,
產生自然語言的摘要 同社區的資訊:
{community_info}

Summary:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input triples, generate the information summary. No pre-amble.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [57]:
def prepare_string(data):
    nodes_str = "Nodes are:\n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}\n"

    rels_str = "Relationships are:\n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}\n"

    return nodes_str + "\n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

process_community(community_info[0])

{'community': '0-3',
 'summary': '本社區的資訊包含以下幾個節點和關係：\n\n節點：\n1. 保險金受益人（Person）\n2. 宣告利率（Concept）\n3. 儲存生息（Financial_strategy）\n4. 要保人（Person）\n5. 本公司（Organization）\n6. 增值回饋分享金（Financial_product），描述為限以購買增額繳清保險金額辦理。\n\n關係：\n1. 增值回饋分享金收益於要保人。\n2. 增值回饋分享金提供給本公司。\n3. 增值回饋分享金收益於保險金受益人。\n4. 增值回饋分享金收益於要保人（重複關係）。\n5. 增值回饋分享金根據宣告利率計算。\n6. 要保人與本公司終止合約。\n7. 本公司處理增值回饋分享金。\n8. 本公司回饋給要保人。\n9. 儲存生息適用於增值回饋分享金。\n10. 儲存生息收益於要保人。\n\n總結來說，增值回饋分享金是一種金融產品，主要限以購買增額繳清保險金額辦理，並主要收益於要保人和保險金受益人。它由本公司提供並處理，並根據宣告利率計算。此外，儲存生息這一金融策略適用於增值回饋分享金並且也收益於要保人。'}

In [58]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

summaries = []
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_community, community): community for community in community_info}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
        summaries.append(future.result())

Processing communities: 100%|██████████| 11/11 [00:03<00:00,  2.90it/s]


In [59]:
summaries

[{'community': '0-12', 'summary': '本契約是一種Agreement，提供了一項金融產品，稱為增額繳清保險金額。'},
 {'community': '1-4',
  'summary': '台灣人壽是一家組織，它提供一款名為「美鑫美利美元利率變動型終身壽險」的產品。此產品包含了「承保範圍」的概念。'},
 {'community': '0-7',
  'summary': '台灣人壽是一家保險公司，該公司提供一款名為「美鑫美利美元利率變動型終身壽險」的產品。這款產品包含了特定的承保範圍。'},
 {'community': '0-15',
  'summary': '在這個社區中，有三個概念節點，分別是「書面」、「電子郵件」和「通知」。其中，「通知」這個概念節點與「書面」和「電子郵件」之間存在方法（METHOD）的關係，也就是說，通知可以通過書面或電子郵件的方式進行。'},
 {'community': '0-22',
  'summary': '有一個與喪葬費用保險金相關的社區。這些資訊點包括：\n\n- 喪葬費用保險金（概念）\n- 未滿15足歲之未成年人（人員）\n- 民國九十九年二月三日（日期）\n\n在這個社區中，喪葬費用保險金與民國九十九年二月三日有關聯。未滿15足歲之未成年人有資格獲得喪葬費用保險金。'},
 {'community': '0-5',
  'summary': '在這個社區中，有三個重要的節點，包括「現金給付」和「保單年度」這兩個概念節點，以及「被保險人」這個人物節點。關係方面，「被保險人」與「保單年度」之間存在「擁有」(HAS) 的關係，而「被保險人」與「現金給付」之間則存在「收到」(RECEIVES) 的關係。'},
 {'community': '0-3',
  'summary': '在這個社區中，有多個節點和關係。\n\n節點包括：\n- 保險金受益人 (Person)\n- 宣告利率 (Concept)\n- 儲存生息 (Financial_strategy)\n- 要保人 (Person)\n- 本公司 (Organization)\n- 增值回饋分享金 (Financial_product)，描述為限以購買增額繳清保險金額辦理\n\n關係包括：\n- 增值回饋分享金會帶來利益給要保人和保

In [60]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

[]